# Robot Demo

This demo is the "everything works" proof:

- Motors (direct motor control via `robot_moves`)
- Drift (wheel-level)
- Horn (meep meep)
- Eyes (RGB via `eyes_lib`)
- TTS (via `tts_lib`)
- Camera snapshot (via `camera_lib`)

If something fails, it prints the error and keeps going.

In [ ]:
# --- Bootstrap: robust repo root + sys.path (works even if cwd is broken) ---
from pathlib import Path
import os
import sys

def safe_start_dir() -> Path:
    try:
        return Path.cwd().resolve()
    except FileNotFoundError:
        home = os.environ.get('HOME')
        if home:
            return Path(home).resolve()
        return Path('/tmp').resolve()

def find_repo_root(start: Path) -> Path:
    p = start.resolve()
    for _ in range(25):
        if (p / 'common').is_dir() and (p / 'lessons').is_dir():
            return p
        if p.parent == p:
            break
        p = p.parent
    raise FileNotFoundError(f'Could not find repo root from start={start}')

START = safe_start_dir()
ROOT = find_repo_root(START)

COMMON_LIB = ROOT / 'common' / 'lib'
LESSONS_LIB = ROOT / 'lessons' / 'lib'

for path in (COMMON_LIB, LESSONS_LIB):
    if str(path) not in sys.path:
        sys.path.insert(0, str(path))

print('START:', START)
print('Repo root:', ROOT)
print('Using common lib:', COMMON_LIB)
print('Using lessons lib:', LESSONS_LIB)
print('Python:', sys.version.split()[0])

In [ ]:
# --- ROS env sanity (avoid broken CycloneDDS config) ---
import os

if os.environ.get('CYCLONEDDS_URI') == 'file:///etc/cyclonedds/config.xml':
    print('Unsetting broken CYCLONEDDS_URI=file:///etc/cyclonedds/config.xml')
    os.environ.pop('CYCLONEDDS_URI', None)

os.environ.setdefault('ROS_DOMAIN_ID', '0')
os.environ.setdefault('RMW_IMPLEMENTATION', 'rmw_cyclonedds_cpp')

print('ROS_DOMAIN_ID =', os.environ.get('ROS_DOMAIN_ID'))
print('RMW_IMPLEMENTATION =', os.environ.get('RMW_IMPLEMENTATION'))
print('CYCLONEDDS_URI =', os.environ.get('CYCLONEDDS_URI'))

## Imports

In [ ]:
import time, importlib

import robot_moves as rm
import eyes_lib
import camera_lib
import tts_lib

importlib.reload(rm)
importlib.reload(eyes_lib)

print('robot_moves:', getattr(rm, '__file__', '?'))
print('eyes_lib:', getattr(eyes_lib, '__file__', '?'))
print('camera_lib:', getattr(camera_lib, '__file__', '?'))
print('tts_lib:', getattr(tts_lib, '__file__', '?'))

bot = rm.RobotMoves(base_speed=220, rate_hz=30)
print('RobotMoves ready. BASE_SPEED=', rm.BASE_SPEED, 'RATE_HZ=', rm.RATE_HZ)

In [ ]:
# Quick safety: stop first
bot.stop()
time.sleep(0.2)
print('Stopped.')

## Eyes (RGB)

In [ ]:
eyes = None
try:
    eyes = eyes_lib.get_eyes()  # default indices (0,1)
    eyes.diagnose()
    print('Eyes ready')
except Exception as e:
    print('Eyes setup failed:', e)
    eyes = None

In [ ]:
if not eyes:
    print('Skipping eyes demo (not available)')
else:
    print('Eyes: set both red, green, blue, then off')
    eyes.set_both(255, 0, 0); time.sleep(0.4)
    eyes.set_both(0, 255, 0); time.sleep(0.4)
    eyes.set_both(0, 0, 255); time.sleep(0.4)
    eyes.off(); time.sleep(0.2)

    print('Eyes: blink white')
    eyes.blink(color=(255,255,255), period_s=0.2, duration_s=1.2)
    print('Eyes demo done')

## TTS

In [ ]:
def speak(text: str):
    if hasattr(tts_lib, 'say'):
        return tts_lib.say(text)
    if hasattr(tts_lib, 'speak'):
        return tts_lib.speak(text)
    if hasattr(tts_lib, 'tts_to_wav'):
        wav = tts_lib.tts_to_wav(text)
        print('tts_to_wav ->', wav)
        return wav
    raise RuntimeError('No say()/speak()/tts_to_wav() found in tts_lib')

try:
    speak('Kia ora team. This is the robot demo.')
    print('TTS done')
except Exception as e:
    print('TTS failed:', e)

## Camera

In [ ]:
from pathlib import Path

out_dir = Path('/tmp/robot_demo')
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / 'snapshot.jpg'

try:
    if hasattr(camera_lib, 'snapshot'):
        camera_lib.snapshot(str(out_path))
        print('Saved snapshot:', out_path)
    elif hasattr(camera_lib, 'get_frame'):
        frame = camera_lib.get_frame()
        try:
            import cv2
            cv2.imwrite(str(out_path), frame)
            print('Saved snapshot:', out_path)
        except Exception as e:
            print('Got frame but could not save (cv2 missing?):', e)
    else:
        print('camera_lib has no snapshot() or get_frame()')
except Exception as e:
    print('Camera failed:', e)

## Movement

In [ ]:
print('Forward / Backward')
bot.forward(0.6)
time.sleep(0.2)
bot.backward(0.6)
time.sleep(0.2)
bot.stop()
print('Done')

In [ ]:
print('Turn left / right')
bot.turn_left(0.5)
time.sleep(0.2)
bot.turn_right(0.5)
time.sleep(0.2)
bot.stop()
print('Done')

In [ ]:
print('Strafe left / right')
bot.left(0.6)
time.sleep(0.2)
bot.right(0.6)
time.sleep(0.2)
bot.stop()
print('Done')

In [ ]:
print('Diagonals')
bot.diagonal_left(0.6)
time.sleep(0.2)
bot.diagonal_right(0.6)
time.sleep(0.2)
bot.stop()
print('Done')

In [ ]:
print('Drift demo (wheel-level, no /cmd_vel)')
try:
    bot.drift_left(seconds=0.9, turn_blend=0.55)
    time.sleep(0.2)
    bot.drift_right(seconds=0.9, turn_blend=0.55)
    time.sleep(0.2)
    bot.stop()
    print('Done')
except Exception as e:
    bot.stop()
    print('Drift failed:', e)

## Horn

In [ ]:
print('Horn: meep meep')
try:
    bot.horn(block=True)
    print('Done')
except Exception as e:
    print('Horn failed:', e)

## Finish

Stop motors + turn eyes off.

In [ ]:
try:
    bot.stop()
except Exception:
    pass

try:
    if eyes:
        eyes.off()
except Exception:
    pass

print('All done')